In [47]:
import re
import pandas as pd
import fitz  # PyMuPDF
import os

def extract_circuit_numbers(pdf_path):
    """
    Extraire tous les numéros de circuit du PDF
    """
    circuit_info = []
    
    # Ouvrir le PDF avec PyMuPDF
    doc = fitz.open(pdf_path)
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        rotation = page.rotation
        
        # Rechercher des patterns comme "7/W0007,COFLRYB-0.35,GY/W"
        for match in re.finditer(r'(\d+)/W\d+,| \bJ\+(\d+)\b', text):
            if match.group(1):  # Cas standard "X/WXXX,"
                circuit_num = match.group(1)
                match_text = match.group(0)
            else:  # Cas "J+XXX"
                circuit_num = match.group(2)
                match_text = match.group(0)
            
            
            # Obtenir les coordonnées du texte pour permettre l'annotation
            # On recherche le texte exact pour obtenir sa position
            text_instances = page.search_for(match.group(0))
            
            if text_instances:
                
                position = text_instances[0]
                print(position)
                circuit_info.append({
                    'page_num': page_num,
                    'circuit_number': circuit_num,
                    'match_text': match_text,
                    'rect': position,
                    'rotation': rotation
                })
    
    doc.close()
    return circuit_info


def clean_part_references(df, column_name):
    """
    Nettoyer les références de pièces en ne gardant que la partie avant le caractère ":"
    
    Args:
        df: DataFrame pandas à nettoyer
        column_name: Nom de la colonne contenant les références à nettoyer
        
    Returns:
        DataFrame avec la colonne nettoyée
    """
    if column_name in df.columns:
        df[column_name] = df[column_name].astype(str).str.split(':').str[0]
    else:
        print(f"La colonne {column_name} n'existe pas dans le DataFrame.")
    
    return df

def match_with_excel(circuit_info, excel_path, circuit_col='Numéro Circuit', sn_col='SN FILS SIMPLE'):
    """
    Correspondre les numéros de circuit avec le fichier Excel
    """
    # Lire le fichier Excel
    df = pd.read_excel(excel_path)
    
    # Convertir la colonne de numéro de circuit en type numérique si nécessaire
    if df[circuit_col].dtype != 'int64':
        df[circuit_col] = pd.to_numeric(df[circuit_col], errors='coerce')
    
    # Ajouter les informations SN FILS SIMPLE à chaque entrée de circuit
    for entry in circuit_info:
        circuit_num = int(entry['circuit_number'])
        
        # Chercher dans Excel
        matching_row = df[df[circuit_col] == circuit_num]
        
        if not matching_row.empty:
            entry['sn_fils_simple'] = str(matching_row.iloc[0][sn_col])
        else:
            entry['sn_fils_simple'] = ""
    
    return circuit_info

def add_annotations_to_pdf(pdf_path, output_path, annotations):
    """
    Ajouter des annotations au PDF existant
    """
    # Ouvrir le PDF
    doc = fitz.open(pdf_path)
    
    # Grouper les annotations par page
    annotations_by_page = {}
    for ann in annotations:
        page_num = ann['page_num']
        if page_num not in annotations_by_page:
            annotations_by_page[page_num] = []
        annotations_by_page[page_num].append(ann)
    
    # Parcourir chaque page et ajouter les annotations
    for page_num in range(len(doc)):
        if page_num in annotations_by_page:
            page = doc[page_num]
            page_rotation = page.rotation
            
            for ann in annotations_by_page[page_num]:
                circuit_num = ann['circuit_number']
                sn_fils = ann['sn_fils_simple']
                rect = ann['rect']  # Coordonnées du texte trouvé
                
                # Calculer la position de l'annotation
                # L'annotation sera placée à droite du texte trouvé
                x0, y0, x1, y1 = rect
                
                # Texte à ajouter
                annotation_text = f"\t\t\t{sn_fils}"
                
                if page_rotation == 0:
                    # Placement standard à droite
                    text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)
                elif page_rotation == 90:
                    # Rotation 90° - placer en dessous
                    text_point = fitz.Point(x0 + (x1 - x0)/2, y1 - 110)
                elif page_rotation == 180:
                    # Rotation 180° - placer à gauche
                    text_point = fitz.Point(x0 - 110, y0 + (y1 - y0)/2)
                elif page_rotation == 270:
                    # Rotation 270° - placer au-dessus
                    text_point = fitz.Point(x0 + (x1 - x0)/2, y0 - 110)
                
                # Ajouter l'annotation (texte en rouge)
                # text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)  # Décalage à droite du texte original
                page.insert_text(
                    text_point,
                    annotation_text,
                    fontsize=10,
                    color=(1, 0, 0),  # Rouge (R,G,B)
                    rotate=page_rotation
                )
    
    # Enregistrer le PDF modifié
    doc.save(output_path)
    doc.close()
    
    return True


def main():
    pdf_path = r"C:\Users\user\Desktop\App bouhamidi\Data\2D0070266P-F0000_SpliceDiagram.pdf"
    excel_path = r"C:\Users\user\Desktop\App bouhamidi\Data\LISTE CIRCUIT PLAFONNIER BELIZ.xlsx"
    # Créer un nom pour le fichier de sortie basé sur le nom original
    base_name = os.path.splitext(pdf_path)[0]
    output_path = f"{base_name}_avec_SN_FILS.pdf"
    
    # Colonnes dans le fichier Excel (à adapter à votre fichier)
    circuit_column = "Wire Internal Name"
    sn_column = "SN FILS SIMPLE"
    
    print(f"Extraction des numéros de circuit du fichier {pdf_path}...")
    circuit_info = extract_circuit_numbers(pdf_path)
    print(f"{len(circuit_info)} numéros de circuit trouvés.")
    
    print(f"Recherche des correspondances dans {excel_path}...")
    matched_info = match_with_excel(circuit_info, excel_path, circuit_column, sn_column)
    
    print(f"Ajout des annotations au PDF...")
    add_annotations_to_pdf(pdf_path, output_path, matched_info)
    
    print("Traitement terminé avec succès!")
    print(f"Résultat enregistré dans: {output_path}")

if __name__ == "__main__":
    main()

Extraction des numéros de circuit du fichier C:\Users\user\Desktop\App bouhamidi\Data\2D0070266P-F0000_SpliceDiagram.pdf...
Rect(5.0, 83.30000305175781, 53.353599548339844, 97.06999969482422)
Rect(386.0400085449219, 329.4654541015625, 416.3565673828125, 339.1979064941406)
Rect(105.54000091552734, 322.3854675292969, 135.8565216064453, 332.117919921875)
Rect(5.0, 83.30000305175781, 53.353599548339844, 97.06999969482422)
Rect(386.0400085449219, 329.4654541015625, 420.2948913574219, 339.1979064941406)
Rect(101.29000091552734, 322.3854675292969, 135.5448455810547, 332.117919921875)
Rect(5.0, 83.30000305175781, 53.353599548339844, 97.06999969482422)
Rect(386.0400085449219, 341.9654541015625, 420.2948913574219, 351.6979064941406)
Rect(101.29000091552734, 334.8854675292969, 135.5448455810547, 344.617919921875)
Rect(5.0, 83.30000305175781, 53.350399017333984, 97.06999969482422)
Rect(386.0400085449219, 341.9654541015625, 420.2948913574219, 351.6979064941406)
Rect(101.29000091552734, 334.88546752

In [58]:
import re
import pandas as pd
import fitz  # PyMuPDF
import os

def extract_circuit_numbers(pdf_path):
    """
    Extraire tous les numéros de circuit du PDF
    """
    circuit_info = []
    circuits_to_skip = set()  # Pour stocker les circuits à ignorer (associés à un J)
    
    # Ouvrir le PDF avec PyMuPDF
    doc = fitz.open(pdf_path)
    
    # Premier passage: identifier tous les numéros de circuit associés à un joint "J"
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        
        # Identifier les joints J et leurs circuits associés pour les ignorer
        for j_match in re.finditer(r'J\d+\s*\n(\d+)', text):
            circuit_to_skip = j_match.group(1)
            circuits_to_skip.add(circuit_to_skip)
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        rotation = page.rotation
        
        # Obtenir les dimensions de la page
        page_width = page.rect.width
        page_height = page.rect.height
        
        # Rechercher des patterns comme "7/W0007,COFLRYB-0.35,GY/W"
        for match in re.finditer(r'(\d+)/W\d+,|J\+(\d+)\b', text):
            if match.group(1):  # Cas standard "X/WXXX,"
                circuit_num = match.group(1)
                match_text = match.group(0)
            else:  # Cas "J+XXX"
                circuit_num = match.group(2)
                match_text = match.group(0)
            
            # Vérifier si ce circuit doit être ignoré
            if circuit_num in circuits_to_skip:
                print(f"Circuit ignoré: {circuit_num} (associé à un joint)")
                continue
            
            # Obtenir les coordonnées du texte pour permettre l'annotation
            text_instances = page.search_for(match.group(0))
            
            if text_instances:
                position = text_instances[0]
                x0, y0, x1, y1 = position
                
                # Déterminer si le circuit est sur la moitié gauche ou droite de la page
                is_left_side = (x0 + x1) / 2 < page_width / 2
                
                circuit_info.append({
                    'page_num': page_num,
                    'circuit_number': circuit_num,
                    'match_text': match_text,
                    'rect': position,
                    'rotation': rotation,
                    'is_left_side': is_left_side,  # Nouveau: indique si le circuit est sur la moitié gauche
                    'page_width': page_width,  # Stocker la largeur de page pour référence
                    'page_height': page_height  # Stocker la hauteur de page pour référence
                })
    
    doc.close()
    return circuit_info

def match_with_excel(circuit_info, excel_path, circuit_col='Numéro Circuit', sn_col='SN FILS SIMPLE'):
    """
    Correspondre les numéros de circuit avec le fichier Excel
    """
    # Lire le fichier Excel
    df = pd.read_excel(excel_path)
    
    # Convertir la colonne de numéro de circuit en type numérique si nécessaire
    if df[circuit_col].dtype != 'int64':
        df[circuit_col] = pd.to_numeric(df[circuit_col], errors='coerce')
    
    # Ajouter les informations SN FILS SIMPLE à chaque entrée de circuit
    for entry in circuit_info:
        try:
            circuit_num = int(entry['circuit_number'])
            
            # Chercher dans Excel
            matching_row = df[df[circuit_col] == circuit_num]
            
            if not matching_row.empty:
                entry['sn_fils_simple'] = str(matching_row.iloc[0][sn_col])
            else:
                entry['sn_fils_simple'] = ""
        except (ValueError, TypeError):
            # Si le circuit_number n'est pas convertible en entier
            entry['sn_fils_simple'] = ""
    
    return circuit_info

def add_annotations_to_pdf(pdf_path, output_path, annotations):
    """
    Ajouter des annotations au PDF existant avec positionnement adapté
    """
    # Ouvrir le PDF
    doc = fitz.open(pdf_path)
    
    # Grouper les annotations par page
    annotations_by_page = {}
    for ann in annotations:
        page_num = ann['page_num']
        if page_num not in annotations_by_page:
            annotations_by_page[page_num] = []
        annotations_by_page[page_num].append(ann)
    
    # Parcourir chaque page et ajouter les annotations
    for page_num in range(len(doc)):
        if page_num in annotations_by_page:
            page = doc[page_num]
            page_rotation = page.rotation
            
            for ann in annotations_by_page[page_num]:
                circuit_num = ann['circuit_number']
                sn_fils = ann['sn_fils_simple']
                rect = ann['rect']
                is_left_side = ann['is_left_side']
                
                # Calculer la position de l'annotation
                x0, y0, x1, y1 = rect
                
                # Texte à ajouter
                annotation_text = f"{sn_fils}"
                
                # Déterminer la position en fonction de:
                # 1. La position du circuit (gauche/droite)
                # 2. La rotation de la page
                
                # Si la page n'est pas tournée (rotation = 0)
                if page_rotation == 0:
                    if is_left_side:
                        # Circuit sur la moitié gauche -> annotation à gauche
                        text_point = fitz.Point(x0 - 90, y0 + (y1 - y0)/2)
                    else:
                        # Circuit sur la moitié droite -> annotation à droite
                        text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)
                
                # Si la page est tournée à 90°
                elif page_rotation == 90:
                    if is_left_side:
                        # Circuit sur la moitié gauche -> annotation en haut
                        text_point = fitz.Point(x0 + (x1 - x0)/2 + 50, y0 - 10)
                    else:
                        # Circuit sur la moitié droite -> annotation en bas
                        text_point = fitz.Point(x0 + (x1 - x0)/2 + 50, y1 + 10)
                
                # Si la page est tournée à 180°
                elif page_rotation == 180:
                    if is_left_side:
                        # Circuit sur la moitié gauche -> annotation à droite
                        text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)
                    else:
                        # Circuit sur la moitié droite -> annotation à gauche
                        text_point = fitz.Point(x0 - 110, y0 + (y1 - y0)/2)
                
                # Si la page est tournée à 270°
                elif page_rotation == 270:
                    if is_left_side:
                        # Circuit sur la moitié gauche -> annotation en bas
                        text_point = fitz.Point(x0 + (x1 - x0)/2, y1 + 90)
                    else:
                        # Circuit sur la moitié droite -> annotation en haut
                        text_point = fitz.Point(x0 + (x1 - x0)/2, y0 - 90)
                
                # Ajouter l'annotation (texte en rouge)
                page.insert_text(
                    text_point,
                    annotation_text,
                    fontsize=10,
                    color=(1, 0, 0),  # Rouge (R,G,B)
                    rotate=page_rotation
                )
    
    # Enregistrer le PDF modifié
    doc.save(output_path)
    doc.close()
    
    return True

def main():
    pdf_path = r"C:\Users\user\Desktop\App bouhamidi\Data\2D0070266P-F0000_SpliceDiagram.pdf"
    excel_path = r"C:\Users\user\Desktop\App bouhamidi\Data\LISTE CIRCUIT PLAFONNIER BELIZ.xlsx"
    # Créer un nom pour le fichier de sortie basé sur le nom original
    base_name = os.path.splitext(pdf_path)[0]
    output_path = f"{base_name}_avec_SN_FILS.pdf"
    
    # Colonnes dans le fichier Excel (à adapter à votre fichier)
    circuit_column = "Wire Internal Name"
    sn_column = "SN FILS SIMPLE"
    
    print(f"Extraction des numéros de circuit du fichier {pdf_path}...")
    circuit_info = extract_circuit_numbers(pdf_path)
    print(f"{len(circuit_info)} numéros de circuit trouvés.")
    
    print(f"Recherche des correspondances dans {excel_path}...")
    matched_info = match_with_excel(circuit_info, excel_path, circuit_column, sn_column)
    
    print(f"Ajout des annotations au PDF...")
    add_annotations_to_pdf(pdf_path, output_path, matched_info)
    
    print("Traitement terminé avec succès!")
    print(f"Résultat enregistré dans: {output_path}")

if __name__ == "__main__":
    main()

Extraction des numéros de circuit du fichier C:\Users\user\Desktop\App bouhamidi\Data\2D0070266P-F0000_SpliceDiagram.pdf...
79 numéros de circuit trouvés.
Recherche des correspondances dans C:\Users\user\Desktop\App bouhamidi\Data\LISTE CIRCUIT PLAFONNIER BELIZ.xlsx...
Ajout des annotations au PDF...
Traitement terminé avec succès!
Résultat enregistré dans: C:\Users\user\Desktop\App bouhamidi\Data\2D0070266P-F0000_SpliceDiagram_avec_SN_FILS.pdf
Ajout des annotations au PDF...
Traitement terminé avec succès!
Résultat enregistré dans: C:\Users\user\Desktop\App bouhamidi\Data\2D0070266P-F0000_SpliceDiagram_avec_SN_FILS.pdf


In [ ]:
#with PN
# import re
# import pandas as pd
# import fitz  # PyMuPDF
# import os

# def extract_page_information(pdf_path):
#     """
#     Extraire pour chaque page:
#     - Les numéros de circuit
#     - Les part numbers (numéros suivis d'une lettre majuscule)
#     - Le SN GROUP en haut de la page
#     """
#     page_info = {}
#     circuits_to_skip = set()
    
#     # Ouvrir le PDF avec PyMuPDF
#     doc = fitz.open(pdf_path)
    
#     # Premier passage: identifier les circuits à ignorer (associés à un J)
#     for page_num in range(len(doc)):
#         page = doc[page_num]
#         text = page.get_text()
        
#         # Identifier les joints J et leurs circuits associés pour les ignorer
#         for j_match in re.finditer(r'J\d+\s*\n(\d+)', text):
#             circuit_to_skip = j_match.group(1)
#             circuits_to_skip.add(circuit_to_skip)
    
#     # Deuxième passage: extraire toutes les informations par page
#     for page_num in range(len(doc)):
#         page = doc[page_num]
#         text = page.get_text()
#         rotation = page.rotation
#         page_width = page.rect.width
#         page_height = page.rect.height
        
#         # Extraire le SN GROUP (texte centré en haut de page, généralement en gras)
#         sn_group = None
#         for sn_match in re.finditer(r'(\d+[A-Z]\d+\s+[A-Z]\d+)', text):
#             # Vérifier s'il est dans le tiers supérieur de la page
#             text_instances = page.search_for(sn_match.group(0))
#             if text_instances:
#                 rect = text_instances[0]
#                 y_pos = rect[1]  # Coordonnée y du haut du texte
#                 if y_pos < page_height / 3:  # Dans le tiers supérieur de la page
#                     sn_group = sn_match.group(0)
#                     break
        
#         # Extraire tous les part numbers (numéros suivis d'une lettre majuscule)
#         part_numbers = []
#         for part_match in re.finditer(r'\b\d+[A-Z]\b', text):
#             part_number = part_match.group(0)
#             if part_number not in part_numbers:  # Éviter les doublons
#                 part_numbers.append(part_number)
        
#         # Informations pour cette page
#         print(f"Page {page_num}: SN GROUP: {sn_group}, Part Numbers: {part_numbers}")
#         page_info[page_num] = {
#             'circuits': [],
#             'part_numbers': part_numbers,
#             'sn_group': sn_group,
#             'rotation': rotation,
#             'width': page_width,
#             'height': page_height
#         }
        
#         # Extraire les numéros de circuit
#         for match in re.finditer(r'(\d+)/W\d+,|J\+(\d+)\b', text):
#             if match.group(1):  # Cas standard "X/WXXX,"
#                 circuit_num = match.group(1)
#                 match_text = match.group(0)
#             else:  # Cas "J+XXX"
#                 circuit_num = match.group(2)
#                 match_text = match.group(0)
            
#             # Vérifier si ce circuit doit être ignoré
#             if circuit_num in circuits_to_skip:
#                 print(f"Circuit ignoré: {circuit_num} (associé à un joint)")
#                 continue
            
#             # Obtenir les coordonnées du texte
#             text_instances = page.search_for(match.group(0))
            
#             if text_instances:
#                 position = text_instances[0]
#                 x0, y0, x1, y1 = position
                
#                 # Déterminer si le circuit est sur la moitié gauche ou droite de la page
#                 is_left_side = (x0 + x1) / 2 < page_width / 2
                
#                 # Ajouter l'information du circuit
#                 page_info[page_num]['circuits'].append({
#                     'circuit_number': circuit_num,
#                     'match_text': match_text,
#                     'rect': position,
#                     'is_left_side': is_left_side
#                 })
    
#     doc.close()
#     return page_info

# def match_with_excel_and_part_numbers(page_info, excel_path, circuit_col='Numéro Circuit', 
#                                      sn_col='SN FILS SIMPLE', sn_group_col='SN GROUP'):
#     """
#     Correspondre les numéros de circuit et part numbers avec le fichier Excel
#     """
#     # Lire le fichier Excel et nettoyer les noms de colonnes
#     df = pd.read_excel(excel_path)
#     df.columns = [str(col).split(':')[0] for col in df.columns]
    
#     # Convertir la colonne de numéro de circuit en type numérique si nécessaire
#     if df[circuit_col].dtype != 'int64':
#         df[circuit_col] = pd.to_numeric(df[circuit_col], errors='coerce')
    
#     # Pour chaque page, associer les informations aux circuits
#     for page_num, info in page_info.items():
#         page_part_numbers = info['part_numbers']
        
#         # Récupérer les colonnes qui correspondent aux part numbers de cette page
#         matching_columns = [col for col in df.columns if col in page_part_numbers]
        
#         if matching_columns:
#             print(f"Page {page_num} - Part numbers trouvés dans les colonnes: {matching_columns}")
            
#             # Chercher les lignes qui ont une valeur "1" dans au moins une de ces colonnes
#             mask = pd.Series(False, index=df.index)
#             for col in matching_columns:
#                 # Convertir en string pour assurer la compatibilité
#                 df[col] = df[col].astype(str)
#                 # Ajouter au masque les lignes où la valeur est "1" ou "1.0"
#                 mask = mask | df[col].isin(["1", "1.0"])
            
#             # Filtrer le DataFrame pour obtenir ces lignes
#             matching_rows = df[mask]
            
#             if not matching_rows.empty:
#                 print(f"Trouvé {len(matching_rows)} lignes correspondantes pour la page {page_num}")
                
#                 # Traiter chaque circuit de cette page
#                 for circuit in info['circuits']:
#                     try:
#                         circuit_num = int(circuit['circuit_number'])
                        
#                         # Chercher dans les lignes filtrées par part number
#                         circuit_match = matching_rows[matching_rows[circuit_col] == circuit_num]
                        
#                         if not circuit_match.empty:
#                             # Utiliser la première correspondance si plusieurs sont trouvées
#                             row = circuit_match.iloc[0]
#                             circuit['sn_fils_simple'] = str(row[sn_col])
#                             circuit['page_sn_group'] = info['sn_group']  # SN GROUP de la page PDF
                            
#                             # Si disponible, ajouter aussi le SN GROUP de l'Excel
#                             if sn_group_col in df.columns:
#                                 circuit['excel_sn_group'] = str(row[sn_group_col])
#                             else:
#                                 circuit['excel_sn_group'] = ""
                            
#                         else:
#                             # Si aucune correspondance, essayer avec toutes les lignes
#                             all_circuit_match = df[df[circuit_col] == circuit_num]
#                             if not all_circuit_match.empty:
#                                 row = all_circuit_match.iloc[0]
#                                 circuit['sn_fils_simple'] = str(row[sn_col])
#                                 circuit['page_sn_group'] = info['sn_group']
#                                 if sn_group_col in df.columns:
#                                     circuit['excel_sn_group'] = str(row[sn_group_col])
#                                 else:
#                                     circuit['excel_sn_group'] = ""
#                             else:
#                                 circuit['sn_fils_simple'] = ""
#                                 circuit['page_sn_group'] = info['sn_group']
#                                 circuit['excel_sn_group'] = ""
                    
#                     except (ValueError, TypeError):
#                         # Si le circuit_number n'est pas convertible en entier
#                         circuit['sn_fils_simple'] = ""
#                         circuit['page_sn_group'] = info['sn_group']
#                         circuit['excel_sn_group'] = ""
#             else:
#                 # Aucune ligne ne correspond aux part numbers, traiter normalement
#                 for circuit in info['circuits']:
#                     try:
#                         circuit_num = int(circuit['circuit_number'])
                        
#                         # Chercher dans Excel sans filtrer par part number
#                         matching_row = df[df[circuit_col] == circuit_num]
                        
#                         if not matching_row.empty:
#                             circuit['sn_fils_simple'] = str(matching_row.iloc[0][sn_col])
#                             circuit['page_sn_group'] = info['sn_group']
#                             if sn_group_col in df.columns:
#                                 circuit['excel_sn_group'] = str(matching_row.iloc[0][sn_group_col])
#                             else:
#                                 circuit['excel_sn_group'] = ""
#                         else:
#                             circuit['sn_fils_simple'] = ""
#                             circuit['page_sn_group'] = info['sn_group']
#                             circuit['excel_sn_group'] = ""
#                     except (ValueError, TypeError):
#                         circuit['sn_fils_simple'] = ""
#                         circuit['page_sn_group'] = info['sn_group']
#                         circuit['excel_sn_group'] = ""
#         else:
#             # Aucun part number correspondant trouvé, traiter normalement
#             print(f"Aucun part number de la page {page_num} ne correspond aux colonnes Excel")
#             for circuit in info['circuits']:
#                 try:
#                     circuit_num = int(circuit['circuit_number'])
                    
#                     # Chercher dans Excel
#                     matching_row = df[df[circuit_col] == circuit_num]
                    
#                     if not matching_row.empty:
#                         circuit['sn_fils_simple'] = str(matching_row.iloc[0][sn_col])
#                         circuit['page_sn_group'] = info['sn_group']
#                         if sn_group_col in df.columns:
#                             circuit['excel_sn_group'] = str(matching_row.iloc[0][sn_group_col])
#                         else:
#                             circuit['excel_sn_group'] = ""
#                     else:
#                         circuit['sn_fils_simple'] = "Non trouvé"
#                         circuit['page_sn_group'] = info['sn_group']
#                         circuit['excel_sn_group'] = ""
#                 except (ValueError, TypeError):
#                     circuit['sn_fils_simple'] = ""
#                     circuit['page_sn_group'] = info['sn_group']
#                     circuit['excel_sn_group'] = ""
    
#     return page_info

# # def add_annotations_to_pdf(pdf_path, output_path, page_info):
# #     """
# #     Ajouter des annotations au PDF existant avec positionnement adapté
# #     """
# #     # Ouvrir le PDF
# #     doc = fitz.open(pdf_path)
    
# #     # Parcourir chaque page et ajouter les annotations
# #     for page_num, info in page_info.items():
# #         page = doc[page_num]
# #         page_rotation = info['rotation']
        
# #         for circuit in info['circuits']:
# #             sn_fils = circuit['sn_fils_simple']
# #             rect = circuit['rect']
# #             is_left_side = circuit['is_left_side']
            
# #             # Ajouter le SN GROUP et SN FILS à l'annotation
# #             annotation_text = f"{sn_fils}"
            
# #             # Ajouter éventuellement le SN GROUP de l'Excel s'il est différent du SN GROUP de la page
# #             if 'excel_sn_group' in circuit and circuit['excel_sn_group'] and circuit['excel_sn_group'] != circuit['page_sn_group']:
# #                 annotation_text += f" ({circuit['excel_sn_group']})"
            
# #             # Calculer la position de l'annotation
# #             x0, y0, x1, y1 = rect
            
# #             # Déterminer la position en fonction de la position du circuit et rotation de la page
# #             if page_rotation == 0:
# #                 if is_left_side:
# #                     text_point = fitz.Point(x0 - 110, y0 + (y1 - y0)/2)
# #                 else:
# #                     text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)
            
# #             elif page_rotation == 90:
# #                 if is_left_side:
# #                     text_point = fitz.Point(x0 + (x1 - x0)/2, y0 - 110)
# #                 else:
# #                     text_point = fitz.Point(x0 + (x1 - x0)/2, y1 + 110)
            
# #             elif page_rotation == 180:
# #                 if is_left_side:
# #                     text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)
# #                 else:
# #                     text_point = fitz.Point(x0 - 110, y0 + (y1 - y0)/2)
            
# #             elif page_rotation == 270:
# #                 if is_left_side:
# #                     text_point = fitz.Point(x0 + (x1 - x0)/2, y1 + 110)
# #                 else:
# #                     text_point = fitz.Point(x0 + (x1 - x0)/2, y0 - 110)
            
# #             # Ajouter l'annotation (texte en rouge)
# #             page.insert_text(
# #                 text_point,
# #                 annotation_text,
# #                 fontsize=10,
# #                 color=(1, 0, 0),  # Rouge (R,G,B)
# #                 rotate=page_rotation
# #             )
    
# #     # Enregistrer le PDF modifié
# #     doc.save(output_path)
# #     doc.close()
    
# #     return True

# def add_annotations_to_pdf(pdf_path, output_path, page_info):
#     """
#     Ajouter des annotations au PDF existant avec positionnement adapté
#     et ajouter le texte entre parenthèses en gras au-dessus
#     """
#     # Ouvrir le PDF
#     doc = fitz.open(pdf_path)
    
#     # Parcourir chaque page et ajouter les annotations
#     for page_num, info in page_info.items():
#         page = doc[page_num]
#         page_rotation = info['rotation']
        
#         for circuit in info['circuits']:
#             sn_fils = circuit['sn_fils_simple']
#             rect = circuit['rect']
#             is_left_side = circuit['is_left_side']
            
#             # Extraire le texte entre parenthèses s'il y en a
#             parenthesis_text = ""
#             if "(" in sn_fils and ")" in sn_fils:
#                 start_idx = sn_fils.find("(")
#                 end_idx = sn_fils.find(")", start_idx)
#                 if start_idx >= 0 and end_idx > start_idx:
#                     parenthesis_text = sn_fils[start_idx + 1:end_idx]
            
#             # Annotations principales (SN FILS SIMPLE)
#             annotation_text = sn_fils
            
#             # Ajouter éventuellement le SN GROUP de l'Excel s'il est différent du SN GROUP de la page
#             if 'excel_sn_group' in circuit and circuit['excel_sn_group'] and circuit['excel_sn_group'] != circuit['page_sn_group']:
#                 annotation_text += f" ({circuit['excel_sn_group']})"
            
#             # Calculer la position de l'annotation
#             x0, y0, x1, y1 = rect
            
#             # Déterminer la position en fonction de la position du circuit et rotation de la page
#             if page_rotation == 0:
#                 if is_left_side:
#                     text_point = fitz.Point(x0 - 110, y0 + (y1 - y0)/2)
#                 else:
#                     text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)
            
#             elif page_rotation == 90:
#                 if is_left_side:
#                     text_point = fitz.Point(x0 + (x1 - x0)/2, y0 - 110)
#                 else:
#                     text_point = fitz.Point(x0 + (x1 - x0)/2, y1 + 110)
            
#             elif page_rotation == 180:
#                 if is_left_side:
#                     text_point = fitz.Point(x1 + 110, y0 + (y1 - y0)/2)
#                 else:
#                     text_point = fitz.Point(x0 - 110, y0 + (y1 - y0)/2)
            
#             elif page_rotation == 270:
#                 if is_left_side:
#                     text_point = fitz.Point(x0 + (x1 - x0)/2, y1 + 110)
#                 else:
#                     text_point = fitz.Point(x0 + (x1 - x0)/2, y0 - 110)
            
#             # Ajouter l'annotation principale (texte en rouge)
#             page.insert_text(
#                 text_point,
#                 annotation_text,
#                 fontsize=10,
#                 color=(1, 0, 0),  # Rouge (R,G,B)
#                 rotate=page_rotation
#             )
            
#             # Si un texte entre parenthèses a été trouvé, l'ajouter au-dessus en gras noir
#             if parenthesis_text:
#                 # Calculer position au-dessus du circuit
#                 if page_rotation == 0:
#                     above_point = fitz.Point((x0 + x1) / 2, y0 - 15)  # 15 points au-dessus
#                 elif page_rotation == 90:
#                     above_point = fitz.Point(x0 - 15, (y0 + y1) / 2)  # 15 points à gauche
#                 elif page_rotation == 180:
#                     above_point = fitz.Point((x0 + x1) / 2, y1 + 15)  # 15 points en dessous
#                 elif page_rotation == 270:
#                     above_point = fitz.Point(x1 + 15, (y0 + y1) / 2)  # 15 points à droite
                
#                 # Ajouter l'annotation en gras noir
#                 page.insert_text(
#                     above_point,
#                     parenthesis_text,
#                     fontsize=10,
#                     color=(0, 0, 0),  # Noir (R,G,B)
#                     fontname="helv-b",  # Police en gras (bold)
#                     rotate=page_rotation
#                 )
    
#     # Enregistrer le PDF modifié
#     doc.save(output_path)
#     doc.close()
    
#     return True

# def main():
#     pdf_path = r"C:\Users\user\Desktop\App bouhamidi\Data\2D0070266P-F0000_SpliceDiagram.pdf"
#     excel_path = r"C:\Users\user\Desktop\App bouhamidi\Data\LISTE CIRCUIT PLAFONNIER BELIZ.xlsx"
#     # Créer un nom pour le fichier de sortie basé sur le nom original
#     base_name = os.path.splitext(pdf_path)[0]
#     output_path = f"{base_name}_avec_SN_FILS.pdf"
    
#     # Colonnes dans le fichier Excel
#     circuit_column = "Wire Internal Name"
#     sn_column = "SN FILS SIMPLE"
#     sn_group_column = "SN GROUP"  # Colonne contenant les SN GROUP dans Excel
    
#     print(f"Extraction des informations du fichier {pdf_path}...")
#     page_info = extract_page_information(pdf_path)
    
#     print(f"Recherche des correspondances dans {excel_path}...")
#     matched_info = match_with_excel_and_part_numbers(
#         page_info, 
#         excel_path, 
#         circuit_column, 
#         sn_column,
#         sn_group_column
#     )
    
#     print(f"Ajout des annotations au PDF...")
#     add_annotations_to_pdf(pdf_path, output_path, matched_info)
    
#     print("Traitement terminé avec succès!")
#     print(f"Résultat enregistré dans: {output_path}")
    
#     # Afficher un résumé
#     total_circuits = sum(len(info['circuits']) for info in page_info.values())
#     total_part_numbers = sum(len(info['part_numbers']) for info in page_info.values())
#     print(f"Total des circuits traités: {total_circuits}")
#     print(f"Total des part numbers trouvés: {total_part_numbers}")

# if __name__ == "__main__":
#     main()